# Communicator Statistics


Sample API:

```
stats = communicator.stats

stats.total.sent.bytes
stats.total.sent.messages
stats.total.received.bytes
stats.total.received.messages

stats.player[3].sent.bytes
stats.player[2].received.messages

stats.tag[Tags.SCATTERV].sent.bytes
stats.tag[42].received.messages
```

In [4]:
import logging
import pprint

import munch
import numpy

from cicada.communicator.interface import Tags
import cicada.communicator
import cicada.encoder
import cicada.additive

logging.basicConfig(level=logging.INFO)

def main(communicator):
    log = cicada.Logger(logging.getLogger(), communicator)
    protocol = cicada.additive.AdditiveProtocol(communicator)

    secret_a = numpy.array(2) if communicator.rank == 0 else None
    secret_b = numpy.array(3.5) if communicator.rank == 1 else None

    log.info(f"Player {communicator.rank} secret a: {secret_a}", src=0)
    log.info(f"Player {communicator.rank} secret b: {secret_b}", src=1)

    a_share = protocol.share(src=0, secret=protocol.encoder.encode(secret_a), shape=())
    b_share = protocol.share(src=1, secret=protocol.encoder.encode(secret_b), shape=())

    log.info(f"Player {communicator.rank} share of a: {a_share}")
    log.info(f"Player {communicator.rank} share of b: {b_share}")

    sum_share = protocol.add(a_share, b_share)

    log.info(f"Player {communicator.rank} share of sum: {sum_share}")

    sum = protocol.encoder.decode(protocol.reveal(sum_share))

    log.info(f"Player {communicator.rank} sum: {sum}")
    
    stats = munch.Munch(communicator.stats)
    #log.info(f"Player {communicator.rank} {stats.total.sent.bytes} {stats.total.received.bytes}")
    #log.info(f"Player {communicator.rank} {stats.tag.keys()}")
    log.info(f"Player {communicator.rank} {stats.tag[Tags.GATHER]}")

cicada.communicator.SocketCommunicator.run(world_size=3, fn=main);

INFO:root:Player 0 secret a: 2
INFO:root:Player 1 secret b: 3.5
INFO:root:Player 0 share of a: cicada.additive.AdditiveArrayShare(storage=8328932880093746165)
INFO:root:Player 1 share of a: cicada.additive.AdditiveArrayShare(storage=10686269101656906323)
INFO:root:Player 2 share of a: cicada.additive.AdditiveArrayShare(storage=17878286165668581698)
INFO:root:Player 0 share of b: cicada.additive.AdditiveArrayShare(storage=6770388348323807086)
INFO:root:Player 1 share of b: cicada.additive.AdditiveArrayShare(storage=12542525889931524057)
INFO:root:Player 2 share of b: cicada.additive.AdditiveArrayShare(storage=17580573909164001347)
INFO:root:Player 0 share of sum: cicada.additive.AdditiveArrayShare(storage=15099321228417553251)
INFO:root:Player 1 share of sum: cicada.additive.AdditiveArrayShare(storage=4782050917878878823)
INFO:root:Player 2 share of sum: cicada.additive.AdditiveArrayShare(storage=17012116001123031488)
INFO:root:Player 0 sum: 5.5
INFO:root:Player 1 sum: 5.5
INFO:root:Pla